In [1]:
from datetime import datetime, timedelta
import re
import sqlite3
import time
import pandas as pd
pd.set_option('display.max_columns', None)
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime

In [2]:
# Create an Engine object
engine = create_engine('sqlite:///fantasy_data.db', echo=False)

chromedriver_path = "C:\\ProgramData\\Anaconda3\\WebDriver\\bin\\chromedriver.exe"
url = 'https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin'

In [3]:
from scraper import Scraper
s = Scraper()

In [8]:
driver = s.cbs_login()

In [51]:
# if that doesn't work
yr = 2025
h_final_stats = f'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/{yr}:p/standard/stats?print_rows=9999'
h_proj_url = 'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/restofseason:p/standard/projections?print_rows=9999'
p_final_stats = f'https://xdl.baseball.cbssports.com/stats/stats-main/all:SP:RP/{yr}:p/standard/stats?print_rows=9999'
p_proj_url = 'https://xdl.baseball.cbssports.com/stats/stats-main/all:SP:RP:P/restofseason:p/standard/projections?print_rows=9999'

In [60]:
stats_type = 'h'

In [61]:
if stats_type == 'h':
    driver.get(h_final_stats)
else:
    driver.get(p_final_stats)

In [62]:
#time.sleep(10)
html = driver.page_source
soup = bs4(html, 'html.parser')

table = soup.find('table', {'class': 'data'})

In [64]:
if stats_type == 'h':
    df = pd.read_html(str(table), header=1, skiprows=1, extract_links='body')[0]
else:
    df = pd.read_html(str(table), header=1, skiprows=1, extract_links='body')[0]
    
df = df.iloc[:-1, 2:]
df.head()

C:\Users\pddnh\AppData\Local\Temp\ipykernel_19360\44259729.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=1, skiprows=1, extract_links='body')[0]


,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank
0,"(Aaron Judge OF • NYY, /players/playerpage/207...","(570, None)","(133, None)","(177, None)","(87, None)","(28, None)","(0, None)","(62, None)","(131, None)","(111, None)","(175, None)","(16, None)","(3, None)","(.310, None)","(.425, None)","(.686, None)","(1, None)"
1,"(Paul Goldschmidt 1B • NYY, /players/playerpag...","(561, None)","(106, None)","(178, None)","(102, None)","(41, None)","(0, None)","(35, None)","(115, None)","(79, None)","(141, None)","(7, None)","(0, None)","(.317, None)","(.404, None)","(.578, None)","(2, None)"
2,"(Freddie Freeman 1B • LAD, /players/playerpage...","(612, None)","(117, None)","(199, None)","(129, None)","(47, None)","(2, None)","(21, None)","(100, None)","(84, None)","(102, None)","(13, None)","(3, None)","(.325, None)","(.407, None)","(.511, None)","(3, None)"
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...","(601, None)","(90, None)","(168, None)","(90, None)","(44, None)","(5, None)","(29, None)","(126, None)","(69, None)","(82, None)","(20, None)","(7, None)","(.280, None)","(.355, None)","(.514, None)","(5, None)"
4,"(Trea Turner SS • PHI, /players/playerpage/213...","(652, None)","(101, None)","(194, None)","(130, None)","(39, None)","(4, None)","(21, None)","(100, None)","(45, None)","(131, None)","(27, None)","(3, None)","(.297, None)","(.343, None)","(.466, None)","(6, None)"


In [65]:
# Apply a lambda function to each column to extract the first element of each tuple
df = df.apply(lambda col: [v[0] if v[1] is None else v for v in col])
df.head()

,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank
0,"(Aaron Judge OF • NYY, /players/playerpage/207...",570,133,177,87,28,0,62,131,111,175,16,3,.310,.425,.686,1
1,"(Paul Goldschmidt 1B • NYY, /players/playerpag...",561,106,178,102,41,0,35,115,79,141,7,0,.317,.404,.578,2
2,"(Freddie Freeman 1B • LAD, /players/playerpage...",612,117,199,129,47,2,21,100,84,102,13,3,.325,.407,.511,3
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...",601,90,168,90,44,5,29,126,69,82,20,7,.280,.355,.514,5
4,"(Trea Turner SS • PHI, /players/playerpage/213...",652,101,194,130,39,4,21,100,45,131,27,3,.297,.343,.466,6


In [66]:
pattern = r'^(?P<FullName>[A-Za-z\.\'\-\s]+) (?P<Positions>[A-Z0-9,]+) • (?P<Team>[A-Z]+)$'

# Extract the player name and ID
df['player'] = df['Player'].apply(lambda x: x[0])
df['id'] = df['Player'].apply(lambda x: x[1])
# Parse id for cbsid
df['cbsid'] = df['id'].apply(lambda x: int(x.replace('/players/playerpage/', '')))
# Use regex to parse out CBSNAME, Pos, and Team
df[['CBSNAME', 'Positions', 'Team']] = df['player'].str.extract(pattern)
df.head()

,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank,player,id,cbsid,CBSNAME,Positions,Team
0,"(Aaron Judge OF • NYY, /players/playerpage/207...",570,133,177,87,28,0,62,131,111,175,16,3,.310,.425,.686,1,Aaron Judge OF • NYY,/players/playerpage/2071264,2071264,Aaron Judge,OF,NYY
1,"(Paul Goldschmidt 1B • NYY, /players/playerpag...",561,106,178,102,41,0,35,115,79,141,7,0,.317,.404,.578,2,Paul Goldschmidt 1B • NYY,/players/playerpage/1765052,1765052,Paul Goldschmidt,1B,NYY
2,"(Freddie Freeman 1B • LAD, /players/playerpage...",612,117,199,129,47,2,21,100,84,102,13,3,.325,.407,.511,3,Freddie Freeman 1B • LAD,/players/playerpage/1630079,1630079,Freddie Freeman,1B,LAD
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...",601,90,168,90,44,5,29,126,69,82,20,7,.280,.355,.514,5,Jose Ramirez 3B • CLE,/players/playerpage/2044495,2044495,Jose Ramirez,3B,CLE
4,"(Trea Turner SS • PHI, /players/playerpage/213...",652,101,194,130,39,4,21,100,45,131,27,3,.297,.343,.466,6,Trea Turner SS • PHI,/players/playerpage/2135249,2135249,Trea Turner,SS,PHI


In [67]:
df

,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank,player,id,cbsid,CBSNAME,Positions,Team
0,"(Aaron Judge OF • NYY, /players/playerpage/207...",570,133,177,87,28,0,62,131,111,175,16,3,.310,.425,.686,1,Aaron Judge OF • NYY,/players/playerpage/2071264,2071264,Aaron Judge,OF,NYY
1,"(Paul Goldschmidt 1B • NYY, /players/playerpag...",561,106,178,102,41,0,35,115,79,141,7,0,.317,.404,.578,2,Paul Goldschmidt 1B • NYY,/players/playerpage/1765052,1765052,Paul Goldschmidt,1B,NYY
2,"(Freddie Freeman 1B • LAD, /players/playerpage...",612,117,199,129,47,2,21,100,84,102,13,3,.325,.407,.511,3,Freddie Freeman 1B • LAD,/players/playerpage/1630079,1630079,Freddie Freeman,1B,LAD
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...",601,90,168,90,44,5,29,126,69,82,20,7,.280,.355,.514,5,Jose Ramirez 3B • CLE,/players/playerpage/2044495,2044495,Jose Ramirez,3B,CLE
4,"(Trea Turner SS • PHI, /players/playerpage/213...",652,101,194,130,39,4,21,100,45,131,27,3,.297,.343,.466,6,Trea Turner SS • PHI,/players/playerpage/2135249,2135249,Trea Turner,SS,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,"(Antony Peguero OF • MIA, /players/playerpage/...",0,0,0,0,0,0,0,0,0,0,0,0,.000,.000,.000,9999,Antony Peguero OF • MIA,/players/playerpage/29077587,29077587,Antony Peguero,OF,MIA
1791,"(Milan Tolentino SS • CLE, /players/playerpage...",0,0,0,0,0,0,0,0,0,0,0,0,.000,.000,.000,9999,Milan Tolentino SS • CLE,/players/playerpage/26615351,26615351,Milan Tolentino,SS,CLE
1792,"(PJ Morlando OF • MIA, /players/playerpage/294...",0,0,0,0,0,0,0,0,0,0,0,0,.000,.000,.000,9999,PJ Morlando OF • MIA,/players/playerpage/29426045,29426045,PJ Morlando,OF,MIA
1793,"(Joey Harris C • WAS, /players/playerpage/2921...",0,0,0,0,0,0,0,0,0,0,0,0,.000,.000,.000,9999,Joey Harris C • WAS,/players/playerpage/2921530,2921530,Joey Harris,C,WAS


In [68]:
if stats_type=='h':
    df['Year'] = yr
    # Be sure to convert all columns to appropriate data types
    for col in ['AB', 'R', 'H', '1B', '2B', '3B', 'HR', 'RBI', 'BB', 'K', 'SB', 'CS', 'Rank']:
        df[col] = df[col].astype(int)
    for col in ['AVG', 'OBP', 'SLG']:
        df[col] = df[col].astype(float)
    df['PA'] = df['AB'] + df['BB']
    df['HBP'] = 0
    df = df[['cbsid', 'CBSNAME', 'Positions', 'Team', 'Year', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR', 'RBI', 'BB', 'K', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'Rank']]
    df[df['AB']>=0].to_csv(f'{s.destination_path}/{yr}-final-stats-{stats_type}.csv', index=False)
    print(f'{yr}-final-stats-{stats_type}.csv saved in {s.destination_path}')
if stats_type=='p':
    df['Year'] = yr
    df['ER'] = df['ERA']/9*df['INNs']
    df.rename(columns={'H':'HA', 'K':'SO', 'S':'SV', 'HD':'HLD'},inplace=True)
    df['SvHld'] = df['SV'] + df['HLD']
    # Be sure to convert all columns to appropriate data types
    for col in ['APP', 'GS', 'QS', 'CG', 'W', 'L', 'S', 'BS', 'HLD', 'SO', 'BB', 'HA', 'ER', 'Rank']:
        df[col] = df[col].astype(int)
    for col in ['INNs', 'ERA', 'WHIP']:
        df[col] = df[col].astype(float)
    df = df[['cbsid', 'CBSNAME', 'Positions', 'Team', 'Year', 'INNs', 'GS', 'QS', 'W', 'L', 'SV', 'HLD', 'SO', 'BB', 'HA', 'ER', 'ERA', 'WHIP', 'Rank']]
    # Save to csv if projected IP > 0
    df[df['INNs']>0].to_csv(f'{s.destination_path}/{yr}-final-stats-{stats_type}.csv', index=False)
    print(f'{yr}-final-stats-{stats_type}.csv saved in {s.destination_path}')

2022-final-stats-h.csv saved in C:\Users\pddnh\Documents\GitHub\xdl\data


In [69]:
driver.quit()